In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [2]:
titanic = pd.read_csv("train.csv")

In [3]:
titanic.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
titanic.drop(columns =["PassengerId", "Name", "Ticket", "Cabin"], inplace = True)

In [5]:
titanic

X = titanic.drop(columns = ["Survived"])
y = titanic["Survived"]

In [6]:
## now the goal is use column transformers on columns age and embarked, we need to impute both 
## then we need convert embarked into OHE
## we need to OHE FOR SEX also
## 

titanic.isnull().sum() ## age has some null values and emabrked has some missinng values as well

Survived      0
Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Fare          0
Embarked      2
dtype: int64

In [45]:
from sklearn.compose import ColumnTransformer
from sklearn.impute   import SimpleImputer
from sklearn.preprocessing  import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_selection import SelectKBest,chi2

In [46]:
X_train,X_test, y_train,y_test = train_test_split(X,y,test_size=0.3)

In [47]:
X_train

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
282,3,male,16.0,0,0,9.5000,S
869,3,male,4.0,1,1,11.1333,S
157,3,male,30.0,0,0,8.0500,S
519,3,male,32.0,0,0,7.8958,S
829,1,female,62.0,0,0,80.0000,NaN
...,...,...,...,...,...,...,...
357,2,female,38.0,0,0,13.0000,S
236,2,male,44.0,1,0,26.0000,S
865,2,female,42.0,0,0,13.0000,S
612,3,female,NaN,1,0,15.5000,Q


In [48]:


X_train.shape

(623, 7)

In [49]:
X_test.shape

(268, 7)

In [50]:

y_train.shape

(623,)

In [51]:
## remeber in pipelines whenever you use column transformer, use it with number for third argument

transfomer1 = ColumnTransformer([
("imputer1", SimpleImputer(),[2]),
("imputer2", SimpleImputer(strategy = "most_frequent"),[6])],
remainder = "passthrough")



In [52]:
transfomer2 = ColumnTransformer([
("OHE", OneHotEncoder(sparse_output=False,handle_unknown='ignore'),[1,6])],
remainder = "passthrough")


In [53]:
## here slice is 0-10, because after encoding ohe we have increased the number of columns 
transfomer3 = ColumnTransformer([
("minmax", MinMaxScaler(),slice(0,10))])

In [54]:
transfomer4 = SelectKBest(score_func=chi2,k=8)

transfomer5 = DecisionTreeClassifier()

In [55]:
pipe = make_pipeline(transfomer1, transfomer2, transfomer3, transfomer4, transfomer5)

In [56]:
X_train

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
282,3,male,16.0,0,0,9.5000,S
869,3,male,4.0,1,1,11.1333,S
157,3,male,30.0,0,0,8.0500,S
519,3,male,32.0,0,0,7.8958,S
829,1,female,62.0,0,0,80.0000,NaN
...,...,...,...,...,...,...,...
357,2,female,38.0,0,0,13.0000,S
236,2,male,44.0,1,0,26.0000,S
865,2,female,42.0,0,0,13.0000,S
612,3,female,NaN,1,0,15.5000,Q


In [57]:
pipe.fit(X_train,y_train)

Pipeline(steps=[('columntransformer-1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('imputer1', SimpleImputer(),
                                                  [2]),
                                                 ('imputer2',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('columntransformer-2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('OHE',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  [1, 6])])),
                ('columntransformer-3',
                 ColumnTransformer(transformers=[('minmax', MinMaxScaler(),
                                                  slice(0, 10, None))])),
                ('selectkbest',
                 SelectKBest(k=8,
                             score_func=<function chi2 at 0x0000021E3C4B5620>)),
                ('decisiontreeclassifier', DecisionTreeClassifier())])

In [58]:
pipe.named_steps

{'columntransformer-1': ColumnTransformer(remainder='passthrough',
                   transformers=[('imputer1', SimpleImputer(), [2]),
                                 ('imputer2',
                                  SimpleImputer(strategy='most_frequent'),
                                  [6])]),
 'columntransformer-2': ColumnTransformer(remainder='passthrough',
                   transformers=[('OHE',
                                  OneHotEncoder(handle_unknown='ignore',
                                                sparse_output=False),
                                  [1, 6])]),
 'columntransformer-3': ColumnTransformer(transformers=[('minmax', MinMaxScaler(), slice(0, 10, None))]),
 'selectkbest': SelectKBest(k=8, score_func=<function chi2 at 0x0000021E3C4B5620>),
 'decisiontreeclassifier': DecisionTreeClassifier()}

In [62]:
predictions = pipe.predict(X_test)

In [63]:
predictions

array([1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0,
       1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0], dtype=int64)

In [65]:
from sklearn.metrics import accuracy_score

accuracy_score(predictions,y_test)

0.667910447761194

In [68]:
## now we need accuracy to be confirmed so I will do validation 

from sklearn.model_selection import cross_val_score

cross_val_score(pipe,X_train,y_train, cv=10, scoring = "accuracy")

array([0.66666667, 0.63492063, 0.6031746 , 0.64516129, 0.58064516,
       0.69354839, 0.51612903, 0.69354839, 0.53225806, 0.66129032])

In [69]:
## so model prediction isnt so bad

cross_val_score(pipe,X_train,y_train, cv=10, scoring = "accuracy").mean()

0.6227342549923195

In [70]:
## taking the pipe to be used in production code 


import pickle 

pickle.dump(pipe,open("pipe.pkl", "wb"))
